[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/bads/blob/master/tutorials/8_nb_ensemble_learning.ipynb) 


# Chapter 8 - Ensemble learning 
We covered classification and regression trees in [Tutorial 5](https://github.com/Humboldt-WI/bads/blob/master/tutorials/5_nb_supervised_learning.ipynb) on supervised learning. Today, we will look into ways for improving the predictive power of tree-based classifiers by *ensemble learning*. Recall that an ensemble is a **composition of multiple base models**. The base models are prediction models themselves. In a homogeneous ensemble, we produce the base models with the same supervised learning algorithm. This is what we will do in this tutorial. Our underlying learning algorithm will be a classification tree. 

The lecture introduced three popular homogeneous ensemble frameworks, bagging, random forest, and boosting. We also learned that the latter framework is often implemented using *Adaboost* or, more recently, *gradient boosting*. The goal of the tutorial is to demonstrate the functioning of these ensemble learning frameworks. To that end, we provide **from scratch implementations of multiple ensemble learners**. Another goal is to empower you to use ensemble algorithms in your work/studies. We pursue this goal by walking you through coding demos of **how to train, tune, and apply ensemble learning algorithms** using `sklearn` and other libraries. 


The outline of the tutorial is as follows:
- Preliminaries
- ...

# Preliminaries
We begin as usual with importing our standard libraries and also our standard modeling data. Since we are also familiar data organization by now, we will also use `sklearn` functionality to partition our data into a training and a test set. 

In [27]:
# Import standard Python libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.model_selection import train_test_split

# Some configuration of the plots we will create later
%matplotlib inline  
plt.rcParams["figure.figsize"] = (12,6)

# Load credit risk data in pre-processed format from GitHub
data_url = 'https://raw.githubusercontent.com/Humboldt-WI/bads/master/data/hmeq_modeling.csv' 
df = pd.read_csv(data_url, index_col="index")

# Pretty printing
from pprint import pprint

# Extract target variable and feature matrix 
X = df.drop(['BAD'], axis=1) 
y = df[['BAD']]

# Split data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888)
print("Remember the shape of our data: ")
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4172, 18) (4172, 1) (1788, 18) (1788, 1)


# Random forest
Even more effective for predictions than decision trees, random forests (RF) are useful and powerful ensemble methods for classifying and regressing data. Essentially, a random set of the features are taken to build decision trees. Trees are also built using *bagged data* which are samples of the data taken with replacement. Many trees are grown (sometimes their maximum depth is also specified) and then all trees vote on the response. Each tree's vote generally has the same weight.


## Random forest from scratch
Though most of the the time, `sklearn`  can be used to implement RF, it is useful to take a look at the inner workings of the algorithm. Given that RF is an extension of the *bagging algorithm*, we will not look into bagging separately but cover bagging as part of RF.

### Helper Functions for Individual Trees

Again, a RF model is a composition of individual decision trees. So let's start with a recap how an  individual trees works for our data:

We face a classification problem. Thus, the goal of growing a tree is to obtain leaf nodes in which the class distribution is pure. Many potential ways to split the data are calculated (eg. we are going to go through all unique values of each column and finding the midpoint between sequential values). For each potential split, we evaluate whether it contributes toward our goal of separating the classes. More specifically, we calculate the **impurity** of the parent node and comparing it with the sum of the impurity scores of the child nodes, which would result from the candidate split. We also weight the child nodes' impurity scores by the number of cases (data points) that enter the left and right child node, respectively. 

There are three major impurity functions: **entropy, gini, and misclassification error**. We use entropy in our example. The split which yields the lowest impurity is chosen and the process is repeated for the new nodes (this is recursion). 

The method of choosing the split that yields the lowest impurity is called the **greedy search method**. The following functions will help the decision tree implement greedy search tactics on the data. The algorithm stops either when purity in each node is reached or when it has reached a maximum depth (max amount of recursions we allow) specified in our function.

Many functions for the tree are not found in Python packages and it is cleaner to write them out first then put them together in our main algorithm. Each function below does a specific action, which will be used in our final tree function at the end. Most functions are just copy & pasted from [Tutorial 5](https://github.com/Humboldt-WI/bads/blob/master/tutorials/5_nb_supervised_learning.ipynb). We will highlight changes as we go along.

In [5]:
def check_purity(y):
    
    'checks if a leaf node is perfectly pure, in other words, if the leaf node contains only one class'
    
    unique_classes = np.unique(y) #count number of classes in section of data

    if len(unique_classes) == 1: #check if the node is pure
        return True
    else:
        return False

In [6]:
def classify_data(y):
    
    'classifies data according to the majority class of each leaf'
    
    unique_classes, counts_unique_classes = np.unique(y, return_counts=True)
    #returns classes and no. of obs per class

    index = counts_unique_classes.argmax() #index of class with most obs
    classification = unique_classes[index] #class chosen for classification which is class with most obs
    
    return classification

In [7]:
def split_data(X, y, split_column, split_value):
    
    'splits data based on specific value, will yield both a split for the features X and target y'
    
    split_column_values = X[:, split_column]

    X_below = X[split_column_values <= split_value] #partitions data according to split values from previous functions
    X_above = X[split_column_values >  split_value]
    
    y_below = y[split_column_values <= split_value]
    y_above = y[split_column_values >  split_value]
    
    return X_below, X_above, y_below, y_above

In [8]:
def calculate_entropy(y):
    
    'calculates entropy for each partition of data'
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum() #probability of each class
    entropy = sum(probabilities * -np.log2(probabilities)) #could replace with Gini impurity or misclassification
     
    return entropy

In [9]:
def calculate_overall_entropy(y_below, y_above): 
    
    'calculates the total entropy after each split'
       
    n = len(y_below) + len(y_above)
    p_data_below = len(y_below) / n
    p_data_above = len(y_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(y_below)
                      + p_data_above * calculate_entropy(y_above))
    
    return overall_entropy

In [10]:
def determine_best_split(X, y, potential_splits):
    
    'selects which split lowered entropy the most'
    
    overall_entropy = 9999 #set arbitrarily high, the function will loop over and replace this with lower impurity values
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            X_below, X_above, y_below, y_above = split_data(X, y, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(y_below, y_above)
            
            #goes through each potential split and only updates if it lowers entropy

            if current_overall_entropy <= overall_entropy: 
                overall_entropy = current_overall_entropy #updates only if lower entropy split found, in the ned this is greedy search
                best_split_column = column_index
                best_split_value = value
            
    
    return best_split_column, best_split_value

#### Random subspace
Recall a distinct feature of RF compared to bagging. The algorithm for growing base model trees in RF does not search for the next best split among all features. Instead, it first draws a random subset of features and then finds the next best split among this subset. This mechanism is called random subspace and was introduced in [Ho's 1998 paper](http://dx.doi.org/10.1109/34.709601). 

Random subspace does not make sense when growing an individual tree. This is why we did not cover the approach in  [Tutorial 5](https://github.com/Humboldt-WI/bads/blob/master/tutorials/5_nb_supervised_learning.ipynb). However, random subspace is a vital ingredient to RF and it is best implemented in function to get potential splits. Therefore, we adjust the implementation of `get_potential_splits()` compared to the [Tutorial 5](https://github.com/Humboldt-WI/bads/blob/master/tutorials/5_nb_supervised_learning.ipynb) version, to ready our code for RF. Note that our updated implementation does not use random subspace by default but can activate it easily by setting the corresponding argument. Thus, our implementation supports both, vanilla trees and trees with random subspace.   

In [11]:
def get_potential_splits(X, random_subspace = None):
    
    'first, takes every unique value of every feature in the feature space, then finds the midpoint between each value'
    'modified to add random_subspace for random forest'
    
    potential_splits = {}
    _, n_columns = X.shape #don't need rows, we choose the column to split on
    # only need second value of .shape which is columns
    column_indices = list(range(n_columns))
    
    if random_subspace and random_subspace <= len(column_indices): #randomly chosen features
        column_indices = random.sample(population=column_indices, k=random_subspace)
    
    for column_index in column_indices:
        potential_splits[column_index] = [] 
        values = X[:, column_index] 
        unique_values = np.unique(values) #get all unique values in each column

        for index in range(len(unique_values)): #all unique feature values
            if index != 0: #skip first value, we need the difference between next values
                current_value = unique_values[index]
                previous_value = unique_values[index - 1] #find a value and the next smallest value
                potential_split = (current_value + previous_value) / 2 #find difference between the two as a potential split
                
                #consider all values which lie between two values as a potential split
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

The tree will now implement the helper functions and display the decision which yielded the best split if printed.

In [12]:
def decision_tree_algorithm(X, y, counter=0, min_samples=2, max_depth=5, random_subspace = None): 

    'same function as in the Decision Tree notebook but now we add random_subspace argument'

    # data preparation
    if counter == 0: # counter tells us how deep the tree is, this is before the tree is initiated
        global COLUMN_HEADERS
        COLUMN_HEADERS = X.columns
        X = X.values #change all to NumPy array for faster calculations
        y = y.values
    else:
        data = X #if we have started the tree, X should already be a NumPy array from the code above 
    
    # base cases
    if (check_purity(y)) or (len(X) < min_samples) or (counter == max_depth):
        classification = classify_data(y)
        
        return classification
    
    # recursive part
    else:    
        counter += 1 #tells us how deep the tree is

        # helper functions 
        potential_splits = get_potential_splits(X, random_subspace) #check for all possible splits ONLY using the random subspace and not all features!
        best_split_column, best_split_value = determine_best_split(X, y, potential_splits) #select best split based on entropy
        X_below, X_above, y_below, y_above = split_data(X, y, best_split_column, best_split_value) #execute best split
        
        # code to explain decisions made by tree to users
        feature_name = COLUMN_HEADERS[best_split_column]
        question = "{} <= {}".format(feature_name, best_split_value) #initiate explanation of split
        sub_tree = {question: []}
        
        # pull answers from tree
        yes_answer = decision_tree_algorithm(X_below, y_below, counter, min_samples, max_depth, random_subspace)
        no_answer = decision_tree_algorithm(X_above, y_above, counter, min_samples, max_depth, random_subspace)
        
        # ensure explanation actually shows useful information
        if yes_answer == no_answer: #if decisions are the same, only display one
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

### Helper functions for the Random Forest Algorithm
Now that we have the infrastructure to build one individual classification tree, we can make use of it to grow a full forest of trees. However, the **strength-diversity trade-off** suggests that a naive combination of base models will not give a powerful ensemble model. Rather, for an ensemble learner to work well, we need base models that display both, strength and diversity. RF fosters diversity among its base models by **randomizing tree growing**. So we grow a forest of random trees. Specifically, the two mechanisms of randomization in RF are **bootstrap sampling** and **random subspace**. To implement the corresponding functionality, we will us the `random` library.  

In [13]:
import random #needed for bagging

Using this library, we can implement our bootstrap sampling mechanism as follows. Remember that a bootstrap sample is nothing but a random sample of the same size as the original data drawn *with replacement*. 

In [ ]:
def bootstrapping(X, y, n_bootstrap):

  'takes a random set of observations of the size n_bootstrap'

    bootstrap_indices = np.random.randint(low=0, high=len(X), size=n_bootstrap) #chooses random indices for the sample
    X_bootstrapped = X.iloc[bootstrap_indices]
    y_bootstrapped = y.iloc[bootstrap_indices]
    
    return X_bootstrapped, y_bootstrapped

Once we have all this infrastructure in place, the actual implementation of the RF algorithm is straightforward. All we need to do is to grow as many base model trees as we like, and make sure we activate **random subspace** when growing the individual member trees from a **bootstrap sample** of the training data. Next to the number of trees and number of features to consider for random subspace, our implementation allows the user to specify the size of the bootstrap sample, which is useful to accelerate the algorithm, and the maximum depth of the tree models. That is quite some flexibility, although professional implementations of the RF algorithm offer many more parameters.   

In [ ]:
def random_forest_algorithm(X, y, n_trees, n_bootstrap, n_features, dt_max_depth):
  'puts the bootstrap sample in the decision tree algorithm with max depth and the random subset of features set, in otherwords, builds the forest tree by tree'

    forest = []
    for i in range(n_trees): #loops for the amount of trees set to be in the forest
        X_bootstrapped, y_bootstrapped = bootstrapping(X, y, n_bootstrap)
        tree = decision_tree_algorithm(X_bootstrapped, y_bootstrapped, max_depth=dt_max_depth, random_subspace=n_features) #creates individual trees
        forest.append(tree)
    
    return forest

Being almost ready to develop an  RF classifier, the only missing bit is some functionality to apply the trained model to new data. We split this functionality into two functions, one for predicting one example with one tree, and a wrapper functions that uses our `predict_example` function and calls is for all base models of our RF. 

In [ ]:
def predict_example(example, tree, counter=0):

  'takes one observation and predicts its class'
    
    if counter == 0 and isinstance(tree, dict) == False: # very shallow tree settings may only vote one way, this first if-statement takes its vote into account
        return tree
    
    else:
        counter += 1
    
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

    
# Gathers all test data
def decision_tree_predictions(test_df, tree):
  
  'applies predict_example to all of the test set'

    predictions = test_df.apply(predict_example, args=(tree,), axis=1)
    return predictions

In [ ]:
def random_forest_predictions(test_df, forest):
    df_predictions = {}
    for i in range(len(forest)):
        column_name = "tree_{}".format(i) #key for dictionary
        predictions = decision_tree_predictions(test_df, tree=forest[i]) #predictions from trees
        df_predictions[column_name] = predictions #insert predictions into dictionary

    df_predictions = pd.DataFrame(df_predictions) #change dictionary to pandas DF
    random_forest_predictions = df_predictions.mode(axis=1)[0] #take mode of predictions over trees for final prediction
    #if there is an even number of predictions, just default to the first value (very unlikely with many trees)
    
    return random_forest_predictions

### Ready, stready, go...
Well done, we have our custom-code RF classifier up and ready. Time to put it into action and produce some credit default risk predictions. Since our implementation is maybe not the most efficient one, let's stick to shallow trees and also set the other hyperparameters such that run times are bearable.   

In [14]:
# Grow RF 
forest = random_forest_algorithm(X, y, n_trees=50, n_bootstrap=800, n_features=8, dt_max_depth=5)

NameError: name 'random_forest_algorithm' is not defined

In [ ]:
# Depict one member tree as rule set
forest[0]

{'LOAN <= -1.081076960402989': [{'MORTDUE <= -0.9988023477260403': [{'JOB_Self <= 0.5': [True,
      {'CLAGE <= 1.319329723705905': [False, True]}]},
    {'DEBTINC <= -0.36610310502357446': [{'JOB_Sales <= 0.5': [False, True]},
      {'YOJ <= -0.5961885016974147': [{'VALUE <= -1.2866632148005883': [True,
          False]},
        {'CLNO <= -0.43208664907294947': [True, False]}]}]}]},
  {'NINQ <= 0.3158144850582817': [{'DEBTINC <= 1.9169228230498017': [False,
      True]},
    {'DELINQcat_1+ <= 0.5': [{'CLAGE <= 0.0485464889016222': [{'CLAGE <= -1.4443818774196413': [True,
          False]},
        False]},
      {'CLNO <= 0.5955631974189916': [{'CLNO <= -0.05347881089170803': [True,
          False]},
        True]}]}]}]}

In [ ]:
# Calculate predictions
predictions = random_forest_predictions(X, forest) #get predictions from forest

In [ ]:
error = np.mean(np.vstack(predictions) == np.array(y)) #check error rate

error

0.8701342281879194

The tree is able to classify a good proportion of the test set well even with only a depth of 3. 

Before moving on with using RF in `sklearn`, here is a **little exercise** for you:
Use your customer implementation of RF to develop a bagging ensemble.

In [15]:
# Yourt task: use the above functions to train a bagging ensemble using decision tress as base model


## Random forest in sklearn
As promised, we will also demonstrate how you would use RF in production environments, that is with some proper implementation under the hood. Luckily, RF is maybe one of the most widely available data science algorithm. You find professional implementations on almost every platform. Obviously, this also includes `sklearn`. 

It turns out that training and using RF in `sklearn`is super easy. Check out this code, which shows all it takes.

In [24]:
from sklearn.ensemble import RandomForestClassifier  # import library
rf = RandomForestClassifier()                        # create model object
rf.fit(X_train, y_train)                             # fit model to training set 
yhat = rf.predict_proba(X_test)                      # obtain test set predictions                   

C:\ProgramData\Anaconda3\envs\adams\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


array([[0.99, 0.01],
       [0.48, 0.52],
       [0.58, 0.42],
       ...,
       [0.58, 0.42],
       [0.87, 0.13],
       [0.89, 0.11]])

And that was it. We promised it is easy, die we not ;)

However, the above demo is simplistic and does not really illustrate how you would use RF in practice. That is mainly because we omit hyperparameters and their tuning. **Using an analytical model with some default parameters is never a good idea.** 

#### Tuning RF hyperparameters using grid search
Random forest is often considered robust toward hyperparameters settings. Still, some tuning may be beneficial. For model selection, we draw on the demos of [Tutorial 7](https://github.com/Humboldt-WI/bads/blob/master/tutorials/7_nb_model_selection.ipynb) and adjust these for our focal task of tuning RF. 

We consider the following hyperparameters. 
<br>
- n_estimators: number of trees(models) in forest (ensemble)<br>
- max_features : maximum features in random subspace<br>

There are a couple of more hyperparameters. Normally, you would not need to tune them but for the sake of completeness, here are some more hyperparameters:
- min_samples_split: minimum number of samples required in leaf node before another split is made. If it is less, this node won't split.<br>
- min_samples_leaf: minimum number of samples required to be at a leaf node.<br>
- max_leaf_nodes: maximum number of leaf nodes in a tree<br>
- criterion: splitting function to use, e.g. gini coefficient<br>
- max_depth: pruning parameter, maximum depth of decision tree<br>
- n_jobs: parallelization of model building<br>
- random_state: this parameter is used to define the random selection. It is used for comparison between various models.<br><br>

If hyperparameters are not specified, they will be set to their default. 

**Remark**: Tuning RF might take a while. If you want to speed things up, consider setting the hyperparameter *max_samples*. It allows you to control the size of the bootstrap sample from which each tree is grown. Read the documentation for more information. Smaller sample sizes accelerate the training.

In [ ]:
from sklearn.model_selection import GridSearchCV
print('Tuning random forest classifier')
rf = RandomForestClassifier(random_state=888, max_samples = 0.5)  # This way, bootstrap sample size will be 50% of the training set

# Define meta-parameter grid of candidate settings
# The following settings are just for illustration
param_grid = {'n_estimators': [100, 200, 500],
              'max_features': [1, 2, 4]
              }

# Set up the grid object specifying the tuning options
gs_rf = GridSearchCV(rf, param_grid, cv=5, scoring='roc_auc', verbose=1)
gs_rf.fit(X_train, y_train)

Let's say we are interested in assessing our RF model in terms of ROC analysis. 

In [ ]:
print("Best CV AUC: %0.4f" % gs_rf.best_score_)
print("Optimal RF meta-parameters:")
print(gs_rf.best_params_)

# Find test set auc of the best random forest classifier
fp_rate, tp_rate, _ = metrics.roc_curve(y_test, gs_rf.predict_proba(X_test)[:, 1])
auc_trace.update( {'rf' : metrics.auc(fp_rate, tp_rate)}) 
print('RF test set AUC: {:.4f}'.format(auc_trace['rf']))

You should see some quite impressive AUC value. Let's plot the ROC curve to appreciate the power of our RF. This also shows how to access the final model from the grid-search results.

In [ ]:
# The plot is not new but note the use of gs_rf.best_estimator_ 
metrics.plot_roc_curve(gs_rf.best_estimator_, X_test, y_test)
plt.plot([0, 1], [0, 1], "r--");

# Boosting
Boosting-type algorithms are based on **two core principles**
1. Develop an ensemble sequentially (i.e., add one model at a time)
2. Let the next model in the chain correct the errors of the current ensemble

We discussed to flavors of boosting in the lecture, the **adaboost algorithm**, which was the first instantiation of a boosting ensemble, and **gradient boosting**, which was proposed in the seminal paper [Greedy Function Approximation: A Gradient Boosting Machine](https://www.jstor.org/stable/2699986?seq=1) by Jerome H. Friedman.

This part of the tutorial follows the same approach as above, we first demonstrate implementing a boosting ensemble from scratch and then showcase its use together with a professional machine learning library. Many derivatives of boosting and libraries exist. Some members of the gradient boosting family include:
- [LightGBM from Microsoft](https://lightgbm.readthedocs.io/en/latest/)
- [Catboost from Yandex](https://catboost.ai/)
- [NGBoost from Stanford ML Group](https://stanfordmlgroup.github.io/projects/ngboost/)

Given wide adoption in practice and academia, we opted for focusing the tutorial to **extreme gradient boosting** or XGB for short. Demos for other popular boosting algorithms are available via the above links. Boosting algorithms that do not follow the gradient boosting principle may be considered somewhat out-of-fashion. We will not cover them here. However, the exercises give you an opportunity to implement your own Adaboost ensemble from scratch. Needless to say, `sklearn` also supports adaboost and other boosting algorithms.    

## Verifying the boosting principle
Is it true really true that one model can *learn* or *correct* the errors of another model as promised by the boosting paradigm? Before diving into cutting-edge gradient boosting, let's convince ourselves of this  fundamental premise of boosting. 

To that end, we use the HMEQ data set and demonstrate that training a model on the errors of a previous model helps to reduce classification error. Since it is common practice to implement boosting using trees as base model, we follow this approach.  

In [1]:
from sklearn import tree

### Model training 
Here we will show the effectiveness of corrective models that *train on errors*. We will first train two models, the first will be for regular predictions. The second will predict which observations the first model misclassifies. We will first run the first prediction on test data, then correct these predictions using the second model.


In [8]:
estimators = []  # list to store the two tree models

In [9]:
# train first classifier
clf = tree.DecisionTreeClassifier(criterion="entropy", min_samples_split=2, max_depth=2) #first classifier
dt = clf.fit(X_train, y_train) #fit the classifier
estimators.append(('first model', dt))

Having obtained our first model, we can calculate that model's residuals on the training set. Let's consider a classification setting. More concretely, let's focus on discrete class predictions. Recall that this is the type of output that we obtain when calling the function `predict()`.  

In [10]:
initial_pred = dt.predict(X_train)  # classify training set using first classifier

Next, we identify misclassified observations and calculate the **classification error **.

Let's produce a binary vector of the same size of the training set in which an entry of `True` indicates that the corresponding training set observation is misclassified by our first tree. The way we design our vector also makes it very easy to compute the classification error. All we need is to find the mean of that vector.

In [ ]:
res = initial_pred != y_train.iloc[:,0] 
print("Classification error of tree #1 is {:.4}".format(res.mean()))
print("Total number of errors tree #1 is {:.4}".format(res.sum()))

On to model #2. Here, we train not on $y$ but a new binary target variable indicating whether model #1 classified an observation correctly. We can think of this new target as $y - \hat{y}$ aka a residual. However, since we train on decisions of a binary outcome, this classifier will predict errors of the first classifier.

In [13]:
clf2 = tree.DecisionTreeClassifier(criterion="gini")  # second tree with different specs
dt_res = clf2.fit(X_train, res)                       # note the new target
estimators.append(('second model', dt_res))           # store second model for later
dt_res

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

Calling the `predict`function of our second corrective model, we obtain a prediction which observations model #1 is likely to misclassify. 

In [14]:
likely_misclassifications = dt_res.predict(X_train) 
print("Based on model #2, we expect model #1 to misclassify {} observations.".format(
    likely_misclassifications.sum()))

721

In [37]:
# Check if classified likely misclassifications are the same as residuals
accuracy_misclassifications = likely_misclassifications == res
accuracy_misclassifications

index
706     True
3647    True
4807    True
1250    True
3748    True
        ... 
5218    True
4060    True
1346    True
3454    True
3582    True
Name: BAD, Length: 4768, dtype: bool

In [39]:
# It seems likely misclassifications are exactly congruent with residuals, so the model does work
accuracy_misclassifications.mean() 

1.0

### Model testing

Now that we have our two models, we will begin using the test data to see if a combination of the two models can reduce the value of the residuals. We will first predict y using X_test.

In [19]:
pred_initial_test = dt.predict(X_test)

In [20]:
res_test = pred_initial_test != y_test.iloc[:,0]
print("Test error of model 1: {:.4}".format(res_test.mean()))

0.14093959731543623

Now we predict for which observations model 1 has likely made an error.

In [21]:
likely_misclassifications_test = dt_res.predict(X_test)
likely_misclassifications_test

array([False, False, False, ...,  True, False, False])

Lastly, we correct the likely misclassifications by simply flipping the predicted (from model 1) class label.

In [22]:
pred_corrected = pd.Series(pred_initial_test)
pred_corrected[likely_misclassifications_test] = ~ pred_corrected[likely_misclassifications_test]

In [23]:
pred_initial_test[likely_misclassifications_test]  # check that they have actually been changed

array([False,  True, False,  True, False, False,  True,  True, False,
        True,  True, False, False, False, False,  True, False,  True,
        True,  True, False, False,  True, False,  True,  True, False,
        True, False,  True, False,  True,  True,  True,  True, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True, False, False,
       False,  True,  True, False, False, False,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True, False,  True, False,
        True,  True,  True,  True, False, False, False, False,  True,
        True,  True, False,  True, False,  True,  True,  True,  True,
       False, False,  True, False,  True,  True, False, False, False,
       False, False,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True, False,  True,  True, False,
        True,  True,

In [24]:
pred_corrected[likely_misclassifications_test]  # all the results are opposite, so this worked!

5        True
13      False
17       True
20      False
26       True
        ...  
1165     True
1175    False
1176     True
1182    False
1189    False
Length: 159, dtype: object

Time for the grand final, did we reduce the test error?

In [25]:
res_corrected = np.array(pred_corrected) != y_test.iloc[:,0]
print("Test error after corrected model 1 by model 2: {:.4}".format(res_corrected.mean()))

0.13003355704697986

Hurray!!!

A lower test error indicates that our process worked. We were able to lower the error on a test set using a second model which focused on identifying misclassified cases. Let's now examine how gradient boosting relies on the same principled approach. 

## Gradient Boosting from scratch
Having convinced ourselves that the boosting idea works in principle, let's revisit the steps of Friedman's gradient boosting machine.

Go through a from scratch implementation following one of the many examples like, e.g., 

- https://towardsdatascience.com/gradient-boosting-in-python-from-scratch-4a3d9077367
- https://medium.com/mlreview/gradient-boosting-from-scratch-1e317ae4587d
- https://github.com/eriklindernoren/ML-From-Scratch/blob/master/mlfromscratch/supervised_learning/gradient_boosting.py
- https://github.com/eriklindernoren/ML-From-Scratch/blob/master/mlfromscratch/supervised_learning/gradient_boosting.py

## Gradient boosting in sklearn 
As said above, gradient boosting is not gradient boosting. Different gradient boosting algorithms exist and different implementations exist of each of those. If you want to use the *original gradient boosting machine* as proposed in [Friedman's 2001 paper](https://www.jstor.org/stable/2699986?seq=1) or his [follow-up paper on stochastic gradient boosting](http://dx.doi.org/10.1016/S0167-9473(01)00065-2), we recommend using the class `sklearn.ensemble.GradientBoostingClassifier`. It offers a lot of flexibility (e.g., hyperparameters) and is[ very well documented](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html). However, for later data sets, you might want to use **extreme gradient boosting (XGB)**, which was explicitly designed for highly scalable gradient boosting. Below, we demonstrate application of XGB to our credit risk data set.

Although the title of the chapter promises a `sklearn` implementation, it is actually common to use the `xgboost` library for training XGB models. To be precise, `sklearn` offers an implementation of XGB but this implementation might not incorporate the latest features. As far as the use is concerned, you will not notice any differences between the `xgboost` library and `sklearn`. However, **you might need to install the `xgboost` library before moving on**.

### Tuning XGB hyperparameters
Our modeling pipeline is exactly as before in the RF example, and in general. We tune hyperparameters using grid-search and cross-validating the training data. Once we determined good hyperparameter settings, we train a XGB model with the corresponding configuration on the entire training set and obtain test set prediction, which we assess using ROC analysis. 

In [ ]:
import xgboost as xgb
    # Setting up the grid of meta-parameters
xgb_param_grid = {
    'colsample_bytree': np.linspace(0.5, 0.9, 5),  # random subspace
    'n_estimators': [100, 200],  # ensemble size or number of gradient steps
    'max_depth': [5, 10],   # max depth of decision trees
    'learning_rate': [0.1, 0.01],  # learning rate
    'early_stopping_rounds': [10]}  # early stopping if no improvement after that many iterations

gs_xgb = GridSearchCV(estimator=xgb.XGBClassifier(), param_grid=xgb_param_grid, scoring='roc_auc', cv=5, verbose=1)
gs_xgb.fit(x_train, y_train)

In [ ]:
print("Best CV AUC: %0.4f" % gs_xgb.best_score_)
print("Optimal XGB meta-parameters:")
print(gs_xgb.best_params_)

# Find test set auc of the best XGB  classifier
fp_rate, tp_rate, _ = metrics.roc_curve(y_test, gs_xgb.predict_proba(x_test)[:, 1])
print('XGB test set AUC with optimal meta-parameters: {:.4f}'.format( metrics.auc(fp_rate, tp_rate) ))

In [ ]:
# The plot is not new but note the use of gs_rf.best_estimator_ 
metrics.plot_roc_curve(gs_rf.best_estimator_, x_test, y_test)
plt.plot([0, 1], [0, 1], "r--");

The XGB models completes our journey through the space of ensemble learning algorithms. We have now covered the maybe most important of-the-shelf machine learning algorithms and you have seen how we can apply them to real-world data. 

In future tutorials, we will go back to other steps in the machine learning pipeline and learn about yet more possibilities and procedures to derive value from data.